In [1]:
import joblib
import pandas as pd
import shap
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
model = joblib.load('final_model.pkl')

# Load the scaler
scaler = joblib.load('scaler.pkl')

# Load the SHAP input data
x_shap = pd.read_csv('x_shap.csv')
y_shap = pd.read_csv('y_shap.csv')

In [3]:
imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_shap)

x_train_scaled = scaler.fit_transform(x_train_imputed)
background = shap.kmeans(x_train_scaled, 250)

explainer = shap.KernelExplainer(model.predict_proba, background)

Using 250 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [4]:
background

In [5]:
def explain_single_prediction(patient_features, feature_names, scaler, imputer, explainer, return_raw=False):

    processed = imputer.transform([patient_features])
    processed = scaler.transform(processed)

    shap_values = explainer.shap_values(processed)[0]

    if return_raw:
        final = [pair[1] for pair in shap_values]
        return final

    class1_values = [pair[1] for pair in shap_values]
    explanations = []
    for f, sv in zip(feature_names, class1_values):
        direction = "" if sv > 0 else "NOT"
        explanations.append(
            f"The feature **{f}** pushes toward {direction} being diabetic."
        )

    return explanations



In [10]:
def predict_diabetes(model, scaler, HighBP, HighChol, HeartDiseaseorAttack, Stroke, Smoker, PhysActivity, DiffWalk, BMI, MentHlth, PhysHlth, GenHlth, Sex, Age, HvyAlcoholConsump):
    new_patient = [[HighBP, HighChol, HeartDiseaseorAttack, Stroke, Smoker, PhysActivity, DiffWalk, BMI, MentHlth, PhysHlth, GenHlth, Sex, Age, HvyAlcoholConsump]]
    new_patient_scaled = scaler.transform(new_patient)
    prediction = model.predict(new_patient_scaled)
    
    if prediction[0] == 1:
        print("The patient is likely diabetic.")
        print(prediction)
    else:
        print("The patient is likely not diabetic.")


In [7]:
prediction = model.predict_proba([patient_input])
print("Predicted probabilities:", prediction)


NameError: name 'patient_input' is not defined

In [11]:
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke", "Smoker", "PhysActivity", "DiffWalk", "BMI", "MentHlth", "PhysHlth", "GenHlth", "Sex", "Age", "HvyAlcoholConsump"]


patient_input = [1,1,1,1,1,1,1,34,0,7,4,0,9,0]
non_diabetic  = [0,0,0,0,1,1,0,25,0,0,3,0,7,0]
explanations = explain_single_prediction(
    scaler=scaler,
    patient_features=non_diabetic,
    feature_names=feature_names,
    imputer=imputer,
    explainer=explainer,
)

for sentence in explanations:
    print("->", sentence)


predict_diabetes(model,scaler,0,0,0,0,1,1,0,25,0,0,3,0,7,0)

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

-> The feature **HighBP** pushes toward NOT being diabetic.
-> The feature **HighChol** pushes toward NOT being diabetic.
-> The feature **HeartDiseaseorAttack** pushes toward NOT being diabetic.
-> The feature **Stroke** pushes toward NOT being diabetic.
-> The feature **Smoker** pushes toward NOT being diabetic.
-> The feature **PhysActivity** pushes toward NOT being diabetic.
-> The feature **DiffWalk** pushes toward NOT being diabetic.
-> The feature **BMI** pushes toward NOT being diabetic.
-> The feature **MentHlth** pushes toward NOT being diabetic.
-> The feature **PhysHlth** pushes toward  being diabetic.
-> The feature **GenHlth** pushes toward NOT being diabetic.
-> The feature **Sex** pushes toward NOT being diabetic.
-> The feature **Age** pushes toward NOT being diabetic.
-> The feature **HvyAlcoholConsump** pushes toward  being diabetic.
The patient is likely not diabetic.


In [12]:
raw = explain_single_prediction(
    scaler=scaler,
    patient_features=patient_input,
    feature_names=feature_names,
    imputer=imputer,
    explainer=explainer,
    return_raw=True
)
print(raw)

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

[np.float64(0.06517264672227341), np.float64(0.049467322416154376), np.float64(0.03070546253873255), np.float64(0.02142661009800826), np.float64(0.0), np.float64(0.0), np.float64(0.014602177183948148), np.float64(0.05269138284274403), np.float64(0.0030260841426054674), np.float64(0.0), np.float64(0.14855800455658164), np.float64(0.0), np.float64(0.00931447887096388), np.float64(0.0041376777769291575)]


# Testing Area

Predicted probabilities: [[4.49492722e-05 9.99955051e-01]]


In [24]:
def detect_unexpected_shap_behavior(feature_names, shap_values, patient_features):
    expected = {      
        "HighBP":"positive","HighChol":"positive","HeartDiseaseorAttack":"positive",
        "Stroke":"positive","Smoker":"positive","PhysActivity":"positive",
        "DiffWalk":"positive","BMI":"positive","MentHlth":"positive",
        "PhysHlth":"positive","GenHlth":"negative","Age":"positive",
        "HvyAlcoholConsump":"positive"
    }
    thresholds = {"BMI":30,"MentHlth":7,"PhysHlth":7,"GenHlth":3,"Age":8}

    issues = []
    for f, sv, v in zip(feature_names, shap_values, patient_features):
        if f == "Sex":
            continue

        # ensure scalars
        sv = float(sv)
        v  = float(v)

        exp = expected.get(f)
        if exp is None:
            continue

        if f in thresholds:               
            high = v >= thresholds[f]
            if   exp == "positive" and high and sv < 0:
                issues.append(f"High {f}={v} reduced risk (SHAP {sv:+.3f})")
            elif exp == "negative" and not high and sv > 0:
                issues.append(f"Low  {f} increased risk (SHAP {sv:+.3f})")
        else:                                   # binary features
            if   exp == "positive" and v == 1 and sv < 0:
                issues.append(f"{f}=1 reduced risk (SHAP {sv:+.3f})")
            elif exp == "negative" and v == 0 and sv > 0:
                issues.append(f"{f}=0 increased risk (SHAP {sv:+.3f})")

    return issues

In [12]:
patient_input = [1,1,1,1,1,1,1,34,0,7,4,0,9,0]

raw = explain_single_prediction(
    scaler=scaler,
    patient_features=patient_input,
    feature_names=feature_names,
    imputer=imputer,
    explainer=explainer,
    return_raw=True
)


explanation = explain_single_prediction(
    scaler=scaler,
    patient_features=patient_input,
    feature_names=feature_names,
    imputer=imputer,
    explainer=explainer
)

print(raw)
print(explanation)

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

[np.float64(0.06616720043413857), np.float64(0.04724640322691629), np.float64(0.031068219551679748), np.float64(0.02202814223688351), np.float64(0.0), np.float64(0.0), np.float64(0.014627072098076305), np.float64(0.05369330306125714), np.float64(0.0014604798790514018), np.float64(0.0), np.float64(0.14906031798214495), np.float64(0.0), np.float64(0.008962121495090992), np.float64(0.004336824445555647)]
['The feature **HighBP** pushes toward  being diabetic.', 'The feature **HighChol** pushes toward  being diabetic.', 'The feature **HeartDiseaseorAttack** pushes toward  being diabetic.', 'The feature **Stroke** pushes toward  being diabetic.', 'The feature **Smoker** pushes toward NOT being diabetic.', 'The feature **PhysActivity** pushes toward NOT being diabetic.', 'The feature **DiffWalk** pushes toward  being diabetic.', 'The feature **BMI** pushes toward  being diabetic.', 'The feature **MentHlth** pushes toward NOT being diabetic.', 'The feature **PhysHlth** pushes toward NOT being

In [13]:

df = pd.read_csv("cdc_diabetes_health_indicators.csv")
positive_df = df[df["Diabetes_binary"] == 1]


In [14]:
import pandas as pd
zdf = df[df["Diabetes_binary"] == 0]
mean_MenthHlth = zdf['MentHlth'].mean()
mean_PhysHlth = zdf['PhysHlth'].mean()
mean_GenHlth = zdf['GenHlth'].mean()
mean_age = zdf['Age'].mean()
print("MenthHelth", mean_MenthHlth)
print("PhysHlth", mean_PhysHlth)
print("GenHlth", mean_GenHlth)
print("Age", mean_age)

MenthHelth 2.978033654859069
PhysHlth 3.6410820119633223
GenHlth 2.3851850834043256
Age 7.814064689878809


In [15]:
import pandas as pd
pdf = df[df["Diabetes_binary"] == 1]
mean_MenthHlth = pdf['MentHlth'].mean()
mean_PhysHlth = pdf['PhysHlth'].mean()
mean_GenHlth = pdf['GenHlth'].mean()
mean_age = pdf['Age'].mean()
print("MenthHelth", mean_MenthHlth)
print("PhysHlth", mean_PhysHlth)
print("GenHlth", mean_GenHlth)
print("Age", mean_age)

MenthHelth 4.461806144966899
PhysHlth 7.954478583149437
GenHlth 3.290980591863294
Age 9.379052792395179


In [16]:
for feature, shap_val, patient_feature in zip(feature_names, raw, patient_input):
    print(feature, shap_val, patient_feature)

HighBP 0.06616720043413857 1
HighChol 0.04724640322691629 1
HeartDiseaseorAttack 0.031068219551679748 1
Stroke 0.02202814223688351 1
Smoker 0.0 1
PhysActivity 0.0 1
DiffWalk 0.014627072098076305 1
BMI 0.05369330306125714 34
MentHlth 0.0014604798790514018 0
PhysHlth 0.0 7
GenHlth 0.14906031798214495 4
Sex 0.0 0
Age 0.008962121495090992 9
HvyAlcoholConsump 0.004336824445555647 0


In [17]:
positive_df.shape

(35346, 22)

In [18]:
first_test=positive_df[:10]

In [19]:
first_test.shape

(10, 22)

In [25]:
for idx, row in first_test.iterrows():
    patient = row[feature_names].tolist()

    expl = explain_single_prediction(
        patient_features = patient,
        feature_names    = feature_names,
        scaler           = scaler,
        imputer          = imputer,
        explainer        = explainer
    )

    raw_sv = explain_single_prediction(
        patient_features = patient,
        feature_names    = feature_names,
        scaler           = scaler,
        imputer          = imputer,
        explainer        = explainer,
        return_raw       = True
    )

    issues = detect_unexpected_shap_behavior(
        feature_names    = feature_names,
        shap_values      = raw_sv,
        patient_features = patient
    )

    print(f"\nPatient {idx}  Input: {patient}")
    for s in expl:
        print("->", s)
    if issues:
        print("Issues:")
        for i in issues:
            print("  ", i)



C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]


Patient 8  Input: [1, 1, 1, 0, 1, 0, 1, 30, 30, 30, 5, 0, 9, 0]
-> The feature **HighBP** pushes toward  being diabetic.
-> The feature **HighChol** pushes toward  being diabetic.
-> The feature **HeartDiseaseorAttack** pushes toward  being diabetic.
-> The feature **Stroke** pushes toward NOT being diabetic.
-> The feature **Smoker** pushes toward NOT being diabetic.
-> The feature **PhysActivity** pushes toward  being diabetic.
-> The feature **DiffWalk** pushes toward  being diabetic.
-> The feature **BMI** pushes toward NOT being diabetic.
-> The feature **MentHlth** pushes toward NOT being diabetic.
-> The feature **PhysHlth** pushes toward NOT being diabetic.
-> The feature **GenHlth** pushes toward  being diabetic.
-> The feature **Sex** pushes toward NOT being diabetic.
-> The feature **Age** pushes toward  being diabetic.
-> The feature **HvyAlcoholConsump** pushes toward NOT being diabetic.
Issues:
   High MentHlth=30.0 reduced risk (SHAP -0.022)
   High PhysHlth=30.0 reduce

...

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]


Patient 34  Input: [1, 1, 0, 0, 1, 0, 0, 24, 0, 0, 2, 0, 12, 0]
-> The feature **HighBP** pushes toward  being diabetic.
-> The feature **HighChol** pushes toward  being diabetic.
-> The feature **HeartDiseaseorAttack** pushes toward NOT being diabetic.
-> The feature **Stroke** pushes toward NOT being diabetic.
-> The feature **Smoker** pushes toward NOT being diabetic.
-> The feature **PhysActivity** pushes toward  being diabetic.
-> The feature **DiffWalk** pushes toward NOT being diabetic.
-> The feature **BMI** pushes toward NOT being diabetic.
-> The feature **MentHlth** pushes toward NOT being diabetic.
-> The feature **PhysHlth** pushes toward  being diabetic.
-> The feature **GenHlth** pushes toward NOT being diabetic.
-> The feature **Sex** pushes toward NOT being diabetic.
-> The feature **Age** pushes toward  being diabetic.
-> The feature **HvyAlcoholConsump** pushes toward  being diabetic.


In [ ]:
import pandas as pd
from tqdm import tqdm

# Make sure tqdm is installed: pip install tqdm

issues_log = []

# Iterate through each positive prediction
for idx, row in tqdm(positive_df.iterrows(), total=len(positive_df), desc="Checking SHAP issues"):
    patient = row[feature_names].tolist()

    # Get SHAP values (raw, numeric form)
    shap_values = explain_single_prediction(
        patient_features = patient,
        feature_names    = feature_names,
        scaler           = scaler,
        imputer          = imputer,
        explainer        = explainer,
        return_raw       = True
    )

    # Detect problematic features
    issues = detect_unexpected_shap_behavior(
        feature_names    = feature_names,
        shap_values      = shap_values,
        patient_features = patient
    )

    # Log only if there are any issues
    if issues:
        issues_log.append({
            'index': idx,
            'problematic_features': issues
        })

# Convert to DataFrame
issue_df = pd.DataFrame(issues_log)

# Save the issues to CSV
issue_df.to_csv("flagged_shap_issues.csv", index=False)

print(f"\n✅ Done! Found {len(issue_df)} samples with problematic features.")


Checking SHAP issues:   0%|                                                                                  | 0/35346 [00:00<?, ?it/s]C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Checking SHAP issues:   0%|                                                                        | 1/35346 [00:00<5:04:36,  1.93it/s]C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Checking SHAP issues:   0%|                                                                        | 2/35346 [00:01<5:07:54,  1.91it/s]C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


...

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Checking SHAP issues:   1%|▉                                                                       | 448/35346 [02:26<58:10, 10.00it/s]C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke", "Smoker", "PhysActivity", "DiffWalk", "BMI", "MentHlth", "PhysHlth", "GenHlth", "Sex", "Age", "HvyAlcoholConsump"]


patient_input = [1,1,1,1,1,1,1,34,0,7,4,0,9,0]
non_diabetic  = [0,0,0,0,1,1,0,25,0,0,3,0,7,0]
explanations = explain_single_prediction(
    scaler=scaler,
    patient_features=patient_input,
    feature_names=feature_names,
    imputer=imputer,
    explainer=explainer,
)

for sentence in explanations:
    print("->", sentence)


predict_diabetes(model,scaler,1,1,1,1,1,1,1,34,0,7,4,0,9,0)

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

-> The feature **HighBP** pushes toward  being diabetic.
-> The feature **HighChol** pushes toward  being diabetic.
-> The feature **HeartDiseaseorAttack** pushes toward  being diabetic.
-> The feature **Stroke** pushes toward  being diabetic.
-> The feature **Smoker** pushes toward NOT being diabetic.
-> The feature **PhysActivity** pushes toward NOT being diabetic.
-> The feature **DiffWalk** pushes toward  being diabetic.
-> The feature **BMI** pushes toward  being diabetic.
-> The feature **MentHlth** pushes toward  being diabetic.
-> The feature **PhysHlth** pushes toward NOT being diabetic.
-> The feature **GenHlth** pushes toward  being diabetic.
-> The feature **Sex** pushes toward NOT being diabetic.
-> The feature **Age** pushes toward  being diabetic.
-> The feature **HvyAlcoholConsump** pushes toward  being diabetic.
The patient is likely diabetic.
[1]


In [30]:
raw_sv = explain_single_prediction(
        patient_features = patient_input,
        feature_names    = feature_names,
        scaler           = scaler,
        imputer          = imputer,
        explainer        = explainer,
        return_raw       = True
    )

exp = explain_single_prediction(
        patient_features = patient_input,
        feature_names    = feature_names,
        scaler           = scaler,
        imputer          = imputer,
        explainer        = explainer
    )

issues = detect_unexpected_shap_behavior(
    feature_names    = feature_names,
    shap_values      = raw_sv,
    patient_features = patient
)



print(exp)
print(issues)

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

['The feature **HighBP** pushes toward  being diabetic.', 'The feature **HighChol** pushes toward  being diabetic.', 'The feature **HeartDiseaseorAttack** pushes toward  being diabetic.', 'The feature **Stroke** pushes toward  being diabetic.', 'The feature **Smoker** pushes toward NOT being diabetic.', 'The feature **PhysActivity** pushes toward NOT being diabetic.', 'The feature **DiffWalk** pushes toward  being diabetic.', 'The feature **BMI** pushes toward  being diabetic.', 'The feature **MentHlth** pushes toward  being diabetic.', 'The feature **PhysHlth** pushes toward NOT being diabetic.', 'The feature **GenHlth** pushes toward  being diabetic.', 'The feature **Sex** pushes toward NOT being diabetic.', 'The feature **Age** pushes toward  being diabetic.', 'The feature **HvyAlcoholConsump** pushes toward  being diabetic.']
['Low  GenHlth increased risk (SHAP +0.149)']
